*** Use Transfer Learning(YamNet) with RandomForest firstly, and secondly use self supervised learning ***

1. Transfer Learning and RandomForest

In [ ]:
!pip uninstall -y librosa resampy lazy_loader

Found existing installation: librosa 0.11.0
Uninstalling librosa-0.11.0:
  Successfully uninstalled librosa-0.11.0
Found existing installation: resampy 0.4.3
Uninstalling resampy-0.4.3:
  Successfully uninstalled resampy-0.4.3
Found existing installation: lazy_loader 0.4
Uninstalling lazy_loader-0.4:
  Successfully uninstalled lazy_loader-0.4


In [ ]:
!pip install librosa==0.10.1 resampy==0.4.3 soundfile==0.12.1

  Using cached resampy-0.4.3-py3-none-any.whl.metadata (3.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.7/253.7 kB 9.9 MB/s eta 0:00:00
Using cached resampy-0.4.3-py3-none-any.whl (3.1 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.5 MB/s eta 0:00:00
  Attempting uninstall: soundfile
    Found existing installation: soundfile 0.13.1
    Uninstalling soundfile-0.13.1:
      Successfully uninstalled soundfile-0.13.1


In [ ]:
# Install necessary libraries (Execute in Google Colab environment)
#!pip install librosa
!pip install scikit-learn
# Download the ESC-50 dataset
!wget https://github.com/karolpiczak/ESC-50/archive/master.zip -O ESC-50.zip
!unzip ESC-50.zip
!mv ESC-50-master ESC-50_data

--2025-11-04 00:15:30--  https://github.com/karolpiczak/ESC-50/archive/master.zip
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/karolpiczak/ESC-50/zip/refs/heads/master [following]
--2025-11-04 00:15:30--  https://codeload.github.com/karolpiczak/ESC-50/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 140.82.116.10
Connecting to codeload.github.com (codeload.github.com)|140.82.116.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘ESC-50.zip’

ESC-50.zip              [           <=>      ] 615.78M  14.5MB/s    in 40s     

2025-11-04 00:16:10 (15.5 MB/s) - ‘ESC-50.zip’ saved [645695005]

Archive:  ESC-50.zip
33c8ce9eb2cf0b1c2f8bcf322eb349b6be34dbb6
   creating: ESC-50-master/
   creating: ESC-50-master/.circleci/
  inflating: ESC-50-

In [ ]:
!pip -q install soundfile tensorflow tensorflow_hub scikit-learn pandas
#!pip -q install librosa
import os, numpy as np, pandas as pd, librosa, soundfile as sf, pathlib
from sklearn.model_selection import GroupKFold, cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report
from tqdm import tqdm

# ESC-50 Meta Data & Autio Data
META_CSV = 'ESC-50_data/meta/esc50.csv'
AUDIO_DIR = 'ESC-50_data/audio'
df = pd.read_csv(META_CSV)
print(df.head(10))

            filename  fold  target         category  esc10  src_file take
0   1-100032-A-0.wav     1       0              dog   True    100032    A
1  1-100038-A-14.wav     1      14   chirping_birds  False    100038    A
2  1-100210-A-36.wav     1      36   vacuum_cleaner  False    100210    A
3  1-100210-B-36.wav     1      36   vacuum_cleaner  False    100210    B
4  1-101296-A-19.wav     1      19     thunderstorm  False    101296    A
5  1-101296-B-19.wav     1      19     thunderstorm  False    101296    B
6  1-101336-A-30.wav     1      30  door_wood_knock  False    101336    A
7  1-101404-A-34.wav     1      34      can_opening  False    101404    A
8   1-103298-A-9.wav     1       9             crow  False    103298    A
9  1-103995-A-30.wav     1      30  door_wood_knock  False    103995    A


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

### Transfer Learing with YamNet
yamnet = hub.load("https://tfhub.dev/google/yamnet/1")

def load_audio_16k_mono(path, target_sr=16000):
    y, sr = sf.read(path, dtype='float32', always_2d=False)
    if y.ndim > 1:
        y = np.mean(y, axis=1)  # mono
    if sr != target_sr:
        y = librosa.resample(y, orig_sr=sr, target_sr=target_sr, res_type="kaiser_fast")
        sr = target_sr
    # YAMNet: -1~1 float32
    y = np.clip(y, -1.0, 1.0).astype(np.float32)
    return y, sr

@tf.function
def yamnet_forward(waveform_1d):
    # waveform_1d: tf.Tensor [n_samples], float32
    scores, embeddings, spectrogram = yamnet(waveform_1d)
    return embeddings  # [frames, 1024]

def yamnet_embedding(path):
    y, sr = load_audio_16k_mono(path, 16000)
    wf = tf.convert_to_tensor(y, dtype=tf.float32)
    embs = yamnet_forward(wf).numpy()           # (T, 1024)
    # Time pooling: mean + std → (2048,)
    feat = np.hstack([embs.mean(axis=0), embs.std(axis=0)])
    return feat

In [ ]:
X_yam, y_label, groups = [], [], []

cache_dir = pathlib.Path('/content/yamnet_cache'); cache_dir.mkdir(exist_ok=True)

for _, row in tqdm(df.iterrows(), total=len(df)):
    wav = os.path.join(AUDIO_DIR, row['filename'])
    npy_path = cache_dir / (row['filename'].replace('.wav', '.npy'))

    if npy_path.exists():
        feat = np.load(npy_path)
    else:
        feat = yamnet_embedding(wav)
        np.save(npy_path, feat)

    X_yam.append(feat)
    y_label.append(row['category'])
    groups.append(row['fold'])

X_yam = np.vstack(X_yam)
y_label = np.array(y_label)
groups = np.array(groups)
print("YAMNet feature shape:", X_yam.shape)


100%|██████████| 2000/2000 [03:44<00:00,  8.92it/s]

YAMNet feature shape: (2000, 2048)


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_enc = le.fit_transform(y_label)

### Install
import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report
import pathlib
### Split ESC-50 data into Train set/Validation Set / Test Set
folds = groups.astype(int)
train_mask = np.isin(folds, [1, 2, 3]) # 1,2,3 fold => Train Set
val_mask   = (folds == 4) # 4 fold => Validation Set
test_mask  = (folds == 5) # 5 fold => Test Set

X_train, y_train = X_yam[train_mask], y_enc[train_mask]
X_val,   y_val   = X_yam[val_mask],   y_enc[val_mask]
X_test0, y_test0 = X_yam[test_mask],  y_enc[test_mask]

print("Train:", X_train.shape, " Val:", X_val.shape, " Test(all):", X_test0.shape)

### Inference Set from the test Set
inference_ratio = 0.2
sss = StratifiedShuffleSplit(n_splits=1, test_size=inference_ratio, random_state=42)
test_idx = np.arange(len(X_test0))
test_keep_idx, inf_idx = next(sss.split(X_test0, y_test0))

X_test,  y_test  = X_test0[test_keep_idx], y_test0[test_keep_idx]
X_infer, y_infer = X_test0[inf_idx],       y_test0[inf_idx]

print("Final Split → Train:", X_train.shape,
      " Val:", X_val.shape,
      " Test:", X_test.shape,
      " Inference:", X_infer.shape)

### Feature Scaling
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_val_s   = scaler.transform(X_val)
X_test_s  = scaler.transform(X_test)
X_inf_s   = scaler.transform(X_infer)

###Train Random Forest Classifier..

clf = RandomForestClassifier( #rf
    n_estimators=600,
    max_depth=None,
    max_features="sqrt",
    class_weight="balanced",
    n_jobs=-1,
    random_state=42
)
### Training: Through it, the internal parameter(clf.extimators_) are made.
train_pred = clf.fit(X_train_s, y_train)

#train_acc  = accuracy_score(y_val,  train_pred)
#train_f1   = f1_score(y_train,  train_pred, average="macro")
#print(f"[YAMNet+RF] Train ACC: {train_acc:.3f}, Macro-F1: {train_f1:.3f}")
#print("\n[Train REPORT]\n",  classification_report(y_train,  train_pred,  target_names=le.classes_))

"""
gkf = GroupKFold(n_splits=5)
accs, f1s = [], []
for (tr_idx, te_idx) in gkf.split(X_yam, y_enc, groups):
    rf.fit(X_yam[tr_idx], y_enc[tr_idx])
    pred = rf.predict(X_yam[te_idx])
    accs.append(accuracy_score(y_enc[te_idx], pred))
    f1s.append(f1_score(y_enc[te_idx], pred, average="macro"))

print(f"[YamNet+RandomF] 5-Fold ACC: {np.mean(accs):.3f} ± {np.std(accs):.3f}")
print(f"[YamNet+RandomF] 5-Fold Macro-F1: {np.mean(f1s):.3f} ± {np.std(f1s):.3f}")
"""
###Validation & Testing
val_pred  = clf.predict(X_val_s)
val_acc  = accuracy_score(y_val,  val_pred)
val_f1   = f1_score(y_val,  val_pred, average="macro")
print(f"[YamNet+RandomF] Validation ACC: {val_acc:.3f}, Macro-F1: {val_f1:.3f}")
print("\n[Validation]\n",  classification_report(y_val,  val_pred,  target_names=le.classes_))

test_pred = clf.predict(X_test_s)
test_acc = accuracy_score(y_test, test_pred)
test_f1  = f1_score(y_test, test_pred, average="macro")
print(f"[YamNet+RandomF] Test ACC: {test_acc:.3f}, Macro-F1: {test_f1:.3f}")
print("\n[Testing]\n", classification_report(y_test, test_pred, target_names=le.classes_))






Train: (1200, 2048)  Val: (400, 2048)  Test(all): (400, 2048)
Final Split → Train: (1200, 2048)  Val: (400, 2048)  Test: (320, 2048)  Inference: (80, 2048)
[YamNet+RandomF] Validation ACC: 0.830, Macro-F1: 0.826

[Validation]
                   precision    recall  f1-score   support

        airplane       0.29      0.25      0.27         8
       breathing       0.67      0.75      0.71         8
  brushing_teeth       0.89      1.00      0.94         8
     can_opening       0.55      0.75      0.63         8
        car_horn       0.73      1.00      0.84         8
             cat       1.00      0.75      0.86         8
        chainsaw       0.89      1.00      0.94         8
  chirping_birds       0.88      0.88      0.88         8
    church_bells       1.00      1.00      1.00         8
        clapping       1.00      1.00      1.00         8
     clock_alarm       0.80      1.00      0.89         8
      clock_tick       0.38      0.38      0.38         8
        coughing  

2. Self Supervised Learning

In [ ]:
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
###. Pseudo Label from Inference set
proba = clf.predict_proba(X_inf_s)
conf = proba.max(axis=1)
pseudo = proba.argmax(axis=1)


In [ ]:
### control precision vs reall
TH = 0.90
keep = conf >= TH
print(f"Selected {keep.sum()}/{len(keep)} pseudo-labeled samples at threshold {TH}")

Selected 2/80 pseudo-labeled samples at threshold 0.9


In [ ]:
#Augment Data
X_aug = np.vstack([X_train_s, X_inf_s[keep]])
y_aug = np.hstack([y_train,  pseudo[keep]])

In [ ]:
# Retrain
clf_pl = RandomForestClassifier(
    n_estimators=800, max_features="sqrt",
    class_weight="balanced", n_jobs=-1, random_state=42
)
clf_pl.fit(X_aug, y_aug)

RandomForestClassifier(class_weight='balanced', n_estimators=800, n_jobs=-1,
                       random_state=42)

In [ ]:
val_pred = clf_pl.predict(X_val_s)
val_acc_pl  = accuracy_score(y_val,  val_pred)
val_f1   = f1_score(y_val,  val_pred, average="macro")
test_pred=clf_pl.predict(X_test_s)
test_acc_pl = accuracy_score(y_test, test_pred)
test_f1   = f1_score(y_test,  test_pred, average="macro")
print(f"[Self-Training] Val ACC={val_acc_pl:.3f} F1={val_f1 : .3f} | Test ACC={test_acc_pl:.3f} F1={test_f1 : .3f}  (ΔVal={val_acc_pl-val_acc:+.3f}, ΔTest={test_acc_pl-test_acc:+.3f})")

[Self-Training] Val ACC=0.840 F1= 0.834 | Test ACC=0.841 F1= 0.834  (ΔVal=+0.010, ΔTest=+0.006)
